In [1]:
# Dependencies
import numpy as np
import pandas as pd

In [2]:
censuses_data = pd.read_csv("./CSV/censuses.csv")
# censuses_data = censuses_data[censuses_data["census_year"] == 1900]
censuses_data.count()

line_number               4090
house_number              4090
street_name               4090
last_name                 4090
given_name                4090
relation                  4090
color_race                4090
sex                       4090
age_at_last_birthday      4090
occupation                4090
own_or_rent               4090
owned_free_or_mortgage    4090
house_or_farm             4090
my_notes                  4090
file_name                 4090
census_year               4090
dtype: int64

In [3]:
# Separate out the occupation which are null/ none (LATER we can test our model again these cases)
occupation_missing_mask = (censuses_data["occupation"].str.contains('None')) | (censuses_data["occupation"].str.contains('Null'))
df_missing_occupation = censuses_data[occupation_missing_mask]

df_missing_occupation.count()

line_number               1389
house_number              1389
street_name               1389
last_name                 1389
given_name                1389
relation                  1389
color_race                1389
sex                       1389
age_at_last_birthday      1389
occupation                1389
own_or_rent               1389
owned_free_or_mortgage    1389
house_or_farm             1389
my_notes                  1389
file_name                 1389
census_year               1389
dtype: int64

In [4]:
# prepare new df where occupation column is All labelled
df_all_occupation = censuses_data[occupation_missing_mask == False]

# include following features that MAY influence occupation field -
# color_race, sex, age_at_last_birthday, occupation
newdata = df_all_occupation.loc[:,["color_race","sex","age_at_last_birthday", "occupation"]]
newdata.count()

color_race              2701
sex                     2701
age_at_last_birthday    2701
occupation              2701
dtype: int64

In [5]:
newdata.groupby(['occupation']).count().sort_values(by=['occupation'])

,color_race,sex,age_at_last_birthday
occupation,,,
Laundress At Home,1,1,1
Waitress At Restaurant,1,1,1
Abstracter At Loan And Investments,1,1,1
Accountant At Light And Power Company,1,1,1
Actor At Traveling Show,2,2,2
Actress At Theatrical Company,1,1,1
Agent (Machines),1,1,1
Agent At Real Estate,2,2,2
Agent At Real Estate,1,1,1


In [6]:
exclude_dictionary = ['at','asst.']

In [7]:
from fuzzywuzzy import fuzz, process

In [8]:
correct_occupations=["Agent","Barber","Baker","Beauty","Cook","Labor","Washwoman","Worker", "Wringer","Yard" ]
# replace Yard by Yardman

In [9]:
def fuzzy_match_occ(occupation):
    global df_occu
    new_name, score = process.extractOne(occupation, correct_occupations)
    if(score > 70):
        # Good match case
        if(new_name in df_occu):
        # entry found in corrected occupation then append it
            if not(occupation in df_occu[new_name]):
                df_occu[new_name].append(occupation)
        else:
            # make entry for the new_name found
            df_occu[new_name] = [occupation,]
    else:
        # Poor match score 
        if not(occupation in df_occu):
            # make entry for the new_name found
            df_occu[occupation] = [occupation,]
    
    return df_occu

In [87]:
def fuzziest_match (inp, choices_inp, scorer = 'all', match_score = 75, results = 10, pandas = 'yes'):
    #Set up
    possibilities = {}
    #Pandas Handling
    if pandas == 'yes':
        choices = choices_inp.to_list()
    else:
        choices = choices_inp
    #Score dictionary for fuzz methods
    score_switch = {
        'all': [fuzz.ratio,fuzz.partial_ratio,fuzz.token_sort_ratio,fuzz.WRatio],
        'ratio': fuzz.ratio,
        'partial': fuzz.partial_ratio,
        'token': fuzz.token_sort_ratio,
        'WRatio': fuzz.WRatio}
    #Score Handler
    if scorer == 'ratio':
        scorer_opt = score_switch['ratio']
    elif scorer == 'partial':
        scorer_opt = score_switch['partial']
    elif scorer == 'token':
        scorer_opt = score_switch['token']
    elif scorer == 'WRatio':
        scorer_opt = score_switch['WRatio']
    elif scorer not in score_switch.keys():
        print(scorer +' is not a valid score option:')
        for key in score_switch:
            print(key)
        raise Exception ('Invalid Scorer')
    #Nested DRY functions
    def inner_fuzz (inp, choices, scorer,score_switch):
        matches = process.extract(
            inp,
            choices,
            limit = results,
            scorer = scorer
            )
        for match in range(0,len(matches)):
            if matches[match] in matches:
                del matches[match]
            elif matches[match][1] < match_score:
                del matches[match]
        return matches
    
    def all_inner_fuzz (inp, choices, score_switch):
        for option in score_switch['all']:
            matches = process.extract(
                inp,
                choices,
                limit = results,
                scorer = option
                )
            for match in range(0,len(matches)):
                print(match)
                if matches[match] in matches:
                    del matches[match]
                elif matches[match][1] < match_score:
                    del matches[match]
        return matches

    #Type handling    
    if type(inp) == list:
        if type(inp[0]) == list:
            for lst in inp:
                for lst_item in lst:
                    #Scorer switch case
                    if scorer == 'all':
                        possibilities[lst_item] = all_inner_fuzz(lst_item,choices,score_switch)
                    else:
                        possibilities[lst_item] = inner_fuzz(lst_item,choices,scorer_opt,score_switch)
            return possibilities
        else:
            for item in inp:
                if scorer == 'all':
                    possibilities[item] = all_inner_fuzz(item,choices,score_switch)
                else:
                    possibilities[item] = inner_fuzz(item,choices,scorer_opt,score_switch)
        return possibilities

In [88]:
fuzziest_match(['cook','chef'],newdata.occupation)

0
1
2
3
4
5


IndexError: list index out of range

In [10]:
df_occu = dict()
data = zip(*newdata['occupation'].apply(fuzzy_match_occ))

In [11]:
for row in df_occu:
    print(row, df_occu[row])
    print('~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~')
    print("Matching similar occupation count is ", len(df_occu[row]))
    print("---------------------------------")

Servant ['Servant']
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
Matching similar occupation count is  1
---------------------------------
Drayman ['Drayman']
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
Matching similar occupation count is  1
---------------------------------
Laundress ['Laundress']
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
Matching similar occupation count is  1
---------------------------------
At School ['At School']
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
Matching similar occupation count is  1
---------------------------------
Labor ['Day Laborer', 'Farm Laborer', 'Railroad Laborer', 'Laborer At Railway Freight', 'Laborer At Buildings', 'Laborer At Wholesale House', 'Laborer At Street Car', 'Laborer At Rice Mill', 'Laborer At Oil Mill', 'Laborer At Cotton Compress', 'Laborer At Packing House', 'Laborer At Market', 'Laborer At Rr Shop', 'Laborer At Street Car Beam', 'Laborer At Transfer Co.', 'Laborer At Cement Plant', 'Laborer At Oil Fields', 'Laborer At Street', 'Laborer At City', 'Laborer At Stre

Bricklayers Helper At Buildings ['Bricklayers Helper At Buildings']
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
Matching similar occupation count is  1
---------------------------------
Junk Dealer At City Dump ['Junk Dealer At City Dump']
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
Matching similar occupation count is  1
---------------------------------
Plaster Helper At Buildings ['Plaster Helper At Buildings']
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
Matching similar occupation count is  1
---------------------------------
Owner At Fruit Stand ['Owner At Fruit Stand']
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
Matching similar occupation count is  1
---------------------------------
Truckdriver At Furniture St. ['Truckdriver At Furniture St.']
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
Matching similar occupation count is  1
---------------------------------
Grave Digger At Cemetary Co ['Grave Digger At Cemetary Co']
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
Matching similar occupation count is  1
---------------------------------
Ow

In [16]:
dict_1 = {'first': 1, 'second': 2}
dict_1.keys().

AttributeError: 'dict_keys' object has no attribute 'values'

In [19]:
        
def fuzzy_match_sort (match_name,choices,match_score=50,all=False,scorer = 'pr'):    
    if scorer == 'pr':
        possibilities = process.extract(
            match_name,
            choices,
            limit=100,
            scorer = fuzz.partial_ratio)
    if all == True:
        return [possible for possible in possibilities if possible[1] > match_score]
    else:
        return [possible for possible in possibilities if possible[1] > match_score][0]

IndentationError: expected an indented block (<ipython-input-19-993452407555>, line 4)